In [28]:
import pandas as pd

In [29]:
df = pd.read_excel(r"C:\Projects\Salary_Prediction_app\Employees.xlsx")

In [30]:
df.head()

,No,First Name,Last Name,Gender,Start Date,Years,Department,Country,Center,Monthly Salary,Annual Salary,Job Rate,Sick Leaves,Unpaid Leaves,Overtime Hours
0,1,Ghadir,Hmshw,Male,2018-04-04,2,Quality Control,Egypt,West,1560,18720,3.0,1,0,183
1,2,Omar,Hishan,Male,2020-05-21,0,Quality Control,Saudi Arabia,West,3247,38964,1.0,0,5,198
2,3,Ailya,Sharaf,Female,2017-09-28,3,Major Mfg Projects,Saudi Arabia,West,2506,30072,2.0,0,3,192
3,4,Lwiy,Qbany,Male,2018-08-14,2,Manufacturing,United Arab Emirates,Main,1828,21936,3.0,0,0,7
4,5,Ahmad,Bikri,Male,2020-03-11,0,Manufacturing,Egypt,Main,970,11640,5.0,0,5,121


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689 entries, 0 to 688
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   No              689 non-null    int64         
 1   First Name      689 non-null    object        
 2   Last Name       689 non-null    object        
 3   Gender          689 non-null    object        
 4   Start Date      689 non-null    datetime64[ns]
 5   Years           689 non-null    int64         
 6   Department      689 non-null    object        
 7   Country         689 non-null    object        
 8   Center          689 non-null    object        
 9   Monthly Salary  689 non-null    int64         
 10  Annual Salary   689 non-null    int64         
 11  Job Rate        689 non-null    float64       
 12  Sick Leaves     689 non-null    int64         
 13  Unpaid Leaves   689 non-null    int64         
 14  Overtime Hours  689 non-null    int64         
dtypes: dat

In [32]:
df.isnull().sum()

No                0
First Name        0
Last Name         0
Gender            0
Start Date        0
Years             0
Department        0
Country           0
Center            0
Monthly Salary    0
Annual Salary     0
Job Rate          0
Sick Leaves       0
Unpaid Leaves     0
Overtime Hours    0
dtype: int64

In [33]:
# Selecting the features and target variable

#Drop rows with missing salary or key features
df.dropna(subset=['Annual Salary','Department','Country','Years','Job Rate'], inplace=True)

# Features selection
features = ['Years', 'Job Rate', 'Department', 'Country', 'Sick Leaves', 'Unpaid Leaves','Overtime Hours']
target = 'Annual Salary'

In [43]:
from sklearn.preprocessing import LabelEncoder


In [44]:
le_dept = LabelEncoder()
le_country = LabelEncoder()

In [45]:
df['Department_enc'] = le_dept.fit_transform(df['Department'])
df['Country_enc'] = le_country.fit_transform(df['Country'])

In [46]:
# Final feature set
X = df[['Years', 'Job Rate', 'Department_enc', 'Country_enc', 'Sick Leaves', 'Unpaid Leaves', 'Overtime Hours']]
y = df['Annual Salary']

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [48]:
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from xgboost import XGBRegressor

# Define models
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Train models
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [49]:
#Evaluate models

from sklearn.metrics import mean_squared_error, r2_score

def evaluate_model(model, name):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name} - MSE: {mse:.2f}, R²: {r2:.2f}")
    return r2

r2_rf = evaluate_model(rf_model, "Random Forest")
r2_xgb = evaluate_model(xgb_model, "XGBoost")


Random Forest - MSE: 108185202.81, R²: -0.23
XGBoost - MSE: 122149288.00, R²: -0.39


In [50]:
#Combine using Voting Regressor


voting_model = VotingRegressor([('rf', rf_model), ('xgb', xgb_model)])
voting_model.fit(X_train, y_train)
r2_voting = evaluate_model(voting_model, "Voting Regressor")


Voting Regressor - MSE: 112853669.25, R²: -0.29


In [51]:
import joblib

best_model = max([(rf_model, r2_rf), (xgb_model, r2_xgb), (voting_model, r2_voting)], key=lambda x: x[1])[0]

# Save model and encoders
joblib.dump(best_model, 'best_salary_model.pkl')
joblib.dump(le_dept, 'label_encoder_dept.pkl')
joblib.dump(le_country, 'label_encoder_country.pkl')



['label_encoder_country.pkl']